In [ ]:
"""
ETL Completo con Base de Datos SQL en Google Colab
Ejemplo con SQLite (incluido por defecto) y opción para PostgreSQL
"""

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ============================================
# CONFIGURACIÓN INICIAL
# ============================================

# Crear base de datos SQLite (o conectar si existe)
conn = sqlite3.connect('etl_database.db')
cursor = conn.cursor()

print("✓ Conexión a base de datos establecida")

In [ ]:
# ============================================
# PASO 1: EXTRACT (Extraer)
# ============================================

def extract_from_csv(file_path):
    """Extraer datos desde archivo CSV"""
    try:
        df = pd.read_csv(file_path)
        print(f"✓ Extraídos {len(df)} registros desde CSV")
        return df
    except FileNotFoundError:
        print("⚠ Archivo no encontrado. Creando datos de ejemplo...")
        return create_sample_data()

def extract_from_sql(query):
    """Extraer datos desde tabla SQL"""
    df = pd.read_sql_query(query, conn)
    print(f"✓ Extraídos {len(df)} registros desde SQL")
    return df

def create_sample_data():
    """Crear datos de ejemplo para demostración"""
    np.random.seed(42)

    data = {
        'id': range(1, 101),
        'fecha': [(datetime.now() - timedelta(days=x)).strftime('%Y-%m-%d')
                  for x in range(100)],
        'producto': np.random.choice(['A', 'B', 'C', 'D'], 100),
        'cantidad': np.random.randint(1, 50, 100),
        'precio': np.random.uniform(10, 1000, 100).round(2),
        'cliente': np.random.choice(['Cliente_' + str(i) for i in range(1, 21)], 100),
        'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 100)
    }

    df = pd.DataFrame(data)
    print(f"✓ Creados {len(df)} registros de ejemplo")
    return df

In [ ]:
# ============================================
# PASO 2: TRANSFORM (Transformar)
# ============================================

def transform_data(df):
    """Aplicar transformaciones a los datos"""
    df_transformed = df.copy()

    # 1. Limpiar datos nulos
    initial_rows = len(df_transformed)
    df_transformed = df_transformed.dropna()
    print(f"  - Eliminadas {initial_rows - len(df_transformed)} filas con valores nulos")

    # 2. Convertir tipos de datos
    df_transformed['fecha'] = pd.to_datetime(df_transformed['fecha'])
    df_transformed['cantidad'] = df_transformed['cantidad'].astype(int)
    df_transformed['precio'] = df_transformed['precio'].astype(float)
    print("  - Tipos de datos convertidos")

    # 3. Crear columnas calculadas
    df_transformed['total_venta'] = (df_transformed['cantidad'] *
                                     df_transformed['precio']).round(2)
    df_transformed['año'] = df_transformed['fecha'].dt.year
    df_transformed['mes'] = df_transformed['fecha'].dt.month
    df_transformed['trimestre'] = df_transformed['fecha'].dt.quarter
    print("  - Columnas calculadas agregadas")

    # 4. Categorizar productos
    def categorizar_venta(total):
        if total < 100:
            return 'Baja'
        elif total < 500:
            return 'Media'
        else:
            return 'Alta'

    df_transformed['categoria_venta'] = df_transformed['total_venta'].apply(categorizar_venta)
    print("  - Categorías de venta asignadas")

    # 5. Filtrar datos anómalos (opcional)
    df_transformed = df_transformed[df_transformed['cantidad'] > 0]
    df_transformed = df_transformed[df_transformed['precio'] > 0]
    print(f"  - Datos filtrados: {len(df_transformed)} registros válidos")

    return df_transformed

def create_aggregated_tables(df):
    """Crear tablas agregadas para análisis"""

    # Agregación por producto
    agg_producto = df.groupby('producto').agg({
        'cantidad': 'sum',
        'total_venta': ['sum', 'mean', 'count']
    }).round(2)
    agg_producto.columns = ['_'.join(col).strip() for col in agg_producto.columns]
    agg_producto = agg_producto.reset_index()

    # Agregación por región
    agg_region = df.groupby('region').agg({
        'total_venta': 'sum',
        'cantidad': 'sum',
        'id': 'count'
    }).round(2)
    agg_region.columns = ['ventas_totales', 'cantidad_total', 'num_transacciones']
    agg_region = agg_region.reset_index()

    # Agregación por fecha (mensual)
    df_temp = df.copy()
    df_temp['periodo'] = df_temp['fecha'].dt.to_period('M')
    agg_mensual = df_temp.groupby('periodo').agg({
        'total_venta': 'sum',
        'id': 'count'
    }).round(2)
    agg_mensual.columns = ['ventas_mes', 'transacciones_mes']
    agg_mensual = agg_mensual.reset_index()
    agg_mensual['periodo'] = agg_mensual['periodo'].astype(str)

    print("✓ Tablas agregadas creadas")
    return agg_producto, agg_region, agg_mensual

In [ ]:
# ============================================
# PASO 3: LOAD (Cargar)
# ============================================

def load_to_sql(df, table_name, if_exists='replace'):
    """Cargar datos a la base de datos SQL"""
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)
    print(f"✓ Cargados {len(df)} registros en tabla '{table_name}'")

def create_indexes():
    """Crear índices para mejorar rendimiento de consultas"""
    indexes = [
        "CREATE INDEX IF NOT EXISTS idx_fecha ON ventas(fecha)",
        "CREATE INDEX IF NOT EXISTS idx_producto ON ventas(producto)",
        "CREATE INDEX IF NOT EXISTS idx_region ON ventas(region)",
        "CREATE INDEX IF NOT EXISTS idx_cliente ON ventas(cliente)"
    ]

    for idx in indexes:
        cursor.execute(idx)

    conn.commit()
    print("✓ Índices creados")


In [ ]:
# ============================================
# PASO 4: VALIDACIÓN Y CALIDAD DE DATOS
# ============================================

def validate_data_quality():
    """Validar la calidad de los datos cargados"""
    print("\n=== VALIDACIÓN DE CALIDAD DE DATOS ===")

    # Contar registros
    count = pd.read_sql_query("SELECT COUNT(*) as total FROM ventas", conn)
    print(f"Total de registros: {count['total'][0]}")

    # Verificar nulos
    null_check = pd.read_sql_query("""
        SELECT
            COUNT(*) as total,
            SUM(CASE WHEN fecha IS NULL THEN 1 ELSE 0 END) as fecha_null,
            SUM(CASE WHEN producto IS NULL THEN 1 ELSE 0 END) as producto_null,
            SUM(CASE WHEN total_venta IS NULL THEN 1 ELSE 0 END) as total_null
        FROM ventas
    """, conn)
    print("\nVerificación de valores nulos:")
    print(null_check)

    # Estadísticas básicas
    stats = pd.read_sql_query("""
        SELECT
            MIN(total_venta) as min_venta,
            MAX(total_venta) as max_venta,
            AVG(total_venta) as promedio_venta,
            SUM(total_venta) as total_ventas
        FROM ventas
    """, conn)
    print("\nEstadísticas de ventas:")
    print(stats)

In [ ]:
# ============================================
# CONSULTAS DE EJEMPLO
# ============================================

def run_example_queries():
    """Ejecutar consultas de ejemplo"""
    print("\n=== CONSULTAS DE EJEMPLO ===")

    # Top 5 productos más vendidos
    query1 = """
        SELECT producto,
               SUM(cantidad) as cantidad_total,
               SUM(total_venta) as ventas_totales
        FROM ventas
        GROUP BY producto
        ORDER BY ventas_totales DESC
        LIMIT 5
    """
    print("\nTop 5 Productos por Ventas:")
    print(pd.read_sql_query(query1, conn))

    # Ventas por región
    query2 = """
        SELECT region,
               COUNT(*) as num_ventas,
               SUM(total_venta) as total
        FROM ventas
        GROUP BY region
        ORDER BY total DESC
    """
    print("\nVentas por Región:")
    print(pd.read_sql_query(query2, conn))

    # Mejores clientes
    query3 = """
        SELECT cliente,
               COUNT(*) as num_compras,
               SUM(total_venta) as total_gastado
        FROM ventas
        GROUP BY cliente
        ORDER BY total_gastado DESC
        LIMIT 10
    """
    print("\nTop 10 Clientes:")
    print(pd.read_sql_query(query3, conn))

In [ ]:
# ============================================
# PROCESO ETL PRINCIPAL
# ============================================

def run_etl_process():
    """Ejecutar el proceso ETL completo"""
    print("="*50)
    print("INICIANDO PROCESO ETL")
    print("="*50)

    # EXTRACT
    print("\n[1/4] EXTRAYENDO DATOS...")
    df_raw = extract_from_csv('ventas.csv')

    # TRANSFORM
    print("\n[2/4] TRANSFORMANDO DATOS...")
    df_transformed = transform_data(df_raw)
    agg_producto, agg_region, agg_mensual = create_aggregated_tables(df_transformed)

    # LOAD
    print("\n[3/4] CARGANDO DATOS...")
    load_to_sql(df_transformed, 'ventas', if_exists='replace')
    load_to_sql(agg_producto, 'ventas_por_producto', if_exists='replace')
    load_to_sql(agg_region, 'ventas_por_region', if_exists='replace')
    load_to_sql(agg_mensual, 'ventas_mensuales', if_exists='replace')
    create_indexes()

    # VALIDATE
    print("\n[4/4] VALIDANDO DATOS...")
    validate_data_quality()

    # QUERIES
    run_example_queries()

    print("\n" + "="*50)
    print("✓ PROCESO ETL COMPLETADO EXITOSAMENTE")
    print("="*50)
